In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 5, Finished, Available, Finished, False)

In [1]:
var_src_folder = "netflix_directors"
var_trg_folder = "netflix_directors"


StatementMeta(, 633de736-66c8-47a3-80da-27d4cfe4a66d, 3, Finished, Available, Finished, False)

In [2]:
print(var_src_folder)

StatementMeta(, 633de736-66c8-47a3-80da-27d4cfe4a66d, 4, Finished, Available, Finished, False)

netflix_directors


In [3]:
df = spark.read.table("dbo.netflix_directors")
display(df)


StatementMeta(, 633de736-66c8-47a3-80da-27d4cfe4a66d, 5, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, e2777bb6-0c07-4d3d-822c-b5a85e96b2bf)

In [1]:
df = spark.read.format("delta")\
         .option("header",True)\
         .option("inferSchema",True)\
         .load("abfss://rg_netflix_project@onelake.dfs.fabric.microsoft.com/bronze.Lakehouse/Tables/dbo/netflix_titles")

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 3, Finished, Available, Finished, False)

In [2]:
display(df)

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 4, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, e840a386-7a78-4c59-9870-9a234c9124f7)

In [4]:
df = df.fillna({"duration_minutes": 0, "duration_seasons":1})

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 6, Finished, Available, Finished, False)

In [6]:
display(df)


StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 8, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 41ea36d4-e66a-4425-802b-233a0e4d51fa)

In [7]:
df = df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
        .withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 9, Finished, Available, Finished, False)

In [8]:
df.printSchema()

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 10, Finished, Available, Finished, False)

root
 |-- duration_minutes: integer (nullable = true)
 |-- duration_seasons: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)



In [10]:
df = df.withColumn("rating",split(col('rating'),'-')[0])
display(df)

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 12, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 8fb29bef-79cc-40fd-8d55-53c82c0dcf23)

In [11]:
df = df.withColumn("type_flag",when(col('type')=='Movie',1)\
                      .when(col('type')=='TV Show',2)\
                      .otherwise(0))
display(df)

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 13, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 3a235179-32bc-4db5-8e8b-96f5e91a8fd5)

In [12]:
from pyspark.sql.window import Window

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 14, Finished, Available, Finished, False)

In [13]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col('duration_minutes').desc())))

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 15, Finished, Available, Finished, False)

In [15]:
display(df)

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 17, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 2da36039-831a-41fb-9892-84861c1a2ecf)

In [16]:
df.createOrReplaceGlobalTempView("temp_view")

StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 18, Finished, Available, Finished, False)

In [21]:
df = spark.read.table("dbo.netflix_directors")

df.createOrReplaceGlobalTempView("global_view")


StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 23, Finished, Available, Finished, False)

In [22]:
df2 = spark.sql("""
    SELECT * FROM global_temp.global_view
""")

display(df2)


StatementMeta(, 3605ecd7-0b31-47e3-881e-ca715f531377, 24, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, b52b9ee1-e9af-4e71-bc5f-965ec9daade9)

In [5]:
df = spark.read.table("dbo.netflix_titles")
display(df)


StatementMeta(, 55f43bb8-d42d-4a32-bea1-3b45ce7a4fd4, 7, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, e2e4f7ea-5816-4431-96a7-da621009cad5)

In [7]:
from pyspark.sql.functions import col, regexp_replace

df = df.withColumn(
    "show_id",
    regexp_replace(col("show_id"), "[^0-9]", "").cast("int")
)


StatementMeta(, 55f43bb8-d42d-4a32-bea1-3b45ce7a4fd4, 9, Finished, Available, Finished, False)

In [8]:
df.printSchema()

StatementMeta(, 55f43bb8-d42d-4a32-bea1-3b45ce7a4fd4, 10, Finished, Available, Finished, True)

root
 |-- duration_minutes: string (nullable = true)
 |-- duration_seasons: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: integer (nullable = true)

